## function

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import  confusion_matrix,multilabel_confusion_matrix
from sklearn.metrics import classification_report
import numpy as np

#
def  TrainTestSplit(data):
    print(data.shape) 
    x_data=data[['r','g','b']]
    y_data=data['classes2']
    print(y_data.unique())
    X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=42)
    print(y_train.unique())
    print(y_test.unique())
    y_train_encoded = to_categorical(y_train,num_classes=5)
    y_test_encoded = to_categorical(y_test,num_classes=5) 
    return X_train, X_test,y_train, y_test,  y_train_encoded,y_test_encoded


def model_predict(model,X,Y,labels):
    # predict
    y_pred = model.predict(X)
    y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels
    # Calculate classification report
    classification_report1 = classification_report(Y, y_pred_classes,labels=labels,output_dict=True)
    # Calculate confusion_matrix
    multi_confusion_matrix1=multilabel_confusion_matrix(Y, y_pred_classes,labels=labels)
    confusion_matrix1= confusion_matrix(Y, y_pred_classes,labels=labels)
    
    return  classification_report1,  confusion_matrix1,multi_confusion_matrix1
 
    
    
def model_summary(classification_report,multi_confusion_matrix,confusion_matrix,classes):
    
    # classification_report to DataFrame
    #required_fields=['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil','macro avg', 'weighted avg']#, 'weighted avg'
    required_fields=['0', '1', '2', '3', '4', 'macro avg', 'weighted avg']
    classification_report_dict={k:v['precision'] for k,v in classification_report.items() if k in required_fields}
    #classification_report_dict2={k:v['support'] for k,v in classification_report.items() if k in required_fields}
    #classification_report_dict.update({'accuracy':classification_report['accuracy']})
    classification_report_df=pd.DataFrame(classification_report_dict,index=['precision']).T
    # classification_report_df.index=['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil','macro avg', 'weighted avg','accuracy']#,
    classification_report_df.index=['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil','macro avg', 'weighted avg']#,
    classification_report_df= classification_report_df.sort_values(by = 'precision', ascending=False)
    # confusion_matrix to  DataFrame
    classes=['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil']
    dict_matrix=dict()
    for i,l in zip(range(multi_confusion_matrix.shape[0]),classes):
        cm=multi_confusion_matrix1[i,:,:]
        df_cm = pd.DataFrame(cm,index= ['other',l], columns=['other',l])
        #confusion_matrix, index=class_names, columns=class_names,
        dict_matrix.update({l:df_cm})
    
    classes=['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil']
    confusion_matrix_df= pd.DataFrame(confusion_matrix,index=classes, columns=classes)
    return classification_report_df, dict_matrix, confusion_matrix_df
   
def plot_model_results(model_history_df):
    acc =model_history_df['accuracy']
    val_acc = model_history_df['val_accuracy']
    loss =model_history_df['loss']
    val_loss =model_history_df['val_loss']

    epochs_range = range(model_history_df.shape[0])
    fig,ax = plt.subplots(1,2,figsize=(20,12))
    
    ax[0].plot(epochs_range, acc, label='Training Accuracy')
    ax[0].plot(epochs_range, val_acc, label='Validation Accuracy')
    ax[0].tick_params(axis='both', which='major', labelsize=40)
    ax[0].tick_params(axis='both', which='minor', labelsize=25) 
    ax[0].legend(loc='lower right')
    ax[0].set_title('Training and Validation Accuracy')

    
    ax[1].plot(epochs_range, loss, label='Training Loss')
    ax[1].plot(epochs_range, val_loss, label='Validation Loss')
    ax[1].tick_params(axis='both', which='major', labelsize=40)
    ax[1].tick_params(axis='both', which='minor', labelsize=25) 
    ax[1].legend(loc='upper right')
    ax[1].set_title('Training and Validation Loss')
    plt.show()
    return fig 



# inport the data df from csv

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

Soil_types_directory= 'C:/Users/user/Google Drive/TCD20/python/final_project/Soil_Export'
data_image=pd.read_csv(os.path.join(Soil_types_directory,'all_image_64.csv'))
data_image_select_image=pd.read_csv(os.path.join(Soil_types_directory,'Selecting_image_64.csv'))
data_image_cleaning=pd.read_csv(os.path.join(Soil_types_directory,'all_image_cleaning_128.csv'))


# from df   fully connected network (FNNs)


### Set up the layers

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV



# Define a function that returns your TensorFlow model
def create_FCNNsmodel():
    model = Sequential()
    model.add(Input(shape=(3,)))
    model.add(Dense(16, activation='tanh'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(16, activation='tanh'))
    model.add(Dense(16, activation='tanh'))
    model.add(Dense(5, activation='softmax'))
    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer , loss='binary_crossentropy', metrics=['accuracy'])
    return model


model=create_FCNNsmodel()
model.summary()
#optimizer = Adam(learning_rate=0.0001)
#model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
#tf.keras.utils.plot_model(model, show_dtype=True, show_shapes=True, show_layer_names=True, to_file='model_1.png')


early_stopping_monitor = EarlyStopping(
    monitor='accuracy',
    min_delta=0,
    patience=5,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True)




## data image all pixel

In [ ]:
from tensorflow.keras.utils import to_categorical
data_image2=data_image.groupby(by=['classes']).sample(frac=0.1, random_state=42)
print(data_image2.shape)
epochs=100
X_train,X_test, y_train, y_test,y_train_encoded,y_test_encoded=  TrainTestSplit(data_image2)

### fit the model

In [ ]:
history=model.fit(X_train, y_train_encoded,
          validation_data=(X_test,y_test_encoded ),
          batch_size=16, epochs=epochs, callbacks=[early_stopping_monitor] 
          , verbose=True)

history_df=pd.DataFrame(history.history)
history_df

### Summary model results

In [ ]:

labels=['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil']
labels=[0,1,2,3,4]

classification_report1,confusion_matrix1,multi_confusion_matrix1=model_predict(model,X_test,y_test,labels)
classification_report_df, dict_matrix, confusion_matrix_df=model_summary(classification_report1,multi_confusion_matrix1,confusion_matrix1,labels)
f=plot_model_results(history_df)
Soil_Export= 'C:/Users/user/Google Drive/TCD20/python/final_project/fig'       
f.savefig(os.path.join(Soil_Export,'soil_image_dl.png'))      


#classification_report_df
print('### Yellow Soil ###')
print(dict_matrix['Yellow Soil'])
print('### Laterite Soil	 ###')
print(dict_matrix['Laterite Soil'])
print('### Black Soil ###')
print(dict_matrix['Black Soil'])
print('### Cinder Soill	 ###')
print(dict_matrix['Cinder Soil'])
print('### Peat Soil	 ###')
print(dict_matrix['Peat Soil'])


print(confusion_matrix_df)
Soil_Export= 'C:/Users/user/Google Drive/TCD20/python/final_project/csv'   
classification_report_df.to_csv(os.path.join(Soil_Export,'soil_image_dl.csv'))
classification_report_df



## data image cleaning

In [ ]:
from tensorflow.keras.utils import to_categorical
data_image2=data_image_cleaning.groupby(by=['classes']).sample(frac=0.01, random_state=42)
print(data_image2.shape)
epochs=100
X_train,X_test, y_train, y_test,y_train_encoded,y_test_encoded=  TrainTestSplit(data_image2)

### fit the model

In [ ]:
history=model.fit(X_train, y_train_encoded,
          validation_data=(X_test,y_test_encoded ),
          batch_size=16, epochs=epochs, callbacks=[early_stopping_monitor] 
          , verbose=True)

history_df=pd.DataFrame(history.history)
history_df

### Summary model results

In [ ]:
classification_report1

In [ ]:
labels=['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil']
labels=[0,1,2,3,4]

classification_report1,confusion_matrix1,multi_confusion_matrix1=model_predict(model,X_test,y_test,labels)
classification_report_df, dict_matrix, confusion_matrix_df=model_summary(classification_report1,multi_confusion_matrix1,confusion_matrix1,labels)
f=plot_model_results(history_df)
Soil_Export= 'C:/Users/user/Google Drive/TCD20/python/final_project/fig'       
f.savefig(os.path.join(Soil_Export,'soil_image_dl_a.png'))      


#classification_report_df
print('### Yellow Soil ###')
print(dict_matrix['Yellow Soil'])
print('### Laterite Soil	 ###')
print(dict_matrix['Laterite Soil'])
print('### Black Soil ###')
print(dict_matrix['Black Soil'])
print('### Cinder Soill	 ###')
print(dict_matrix['Cinder Soil'])
print('### Peat Soil	 ###')
print(dict_matrix['Peat Soil'])


print(confusion_matrix_df)
Soil_Export= 'C:/Users/user/Google Drive/TCD20/python/final_project/csv'   
classification_report_df.to_csv(os.path.join(Soil_Export,'soil_image_dl_a.csv'))
classification_report_df


## optimization of batch and epochs
### fit the model

In [ ]:
#############################################################################################################
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV


# Define a function that returns your TensorFlow model
def create_FCNNsmodel():
    model = Sequential()
    model.add(Input(shape=(3,)))
    model.add(Dense(16, activation='tanh'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(16, activation='tanh'))
    model.add(Dense(16, activation='tanh'))
    model.add(Dense(5, activation='softmax'))
    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer , loss='binary_crossentropy', metrics=['accuracy'])
    return model

keras_model = KerasClassifier(reate_FCNNsmodel)
# Define the hyperparameter grid for the grid search
param_grid = {
    'batch_size': [8,10,16],
    'epochs': [90,95,100,105,110,120]   
     }

# Perform grid search with cross-validation and include validation data and callbacks
grid_search = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=5) 
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',  # Metric to monitor (e.g., validation loss)
    patience=3,  # Number of epochs to wait for improvement
    restore_best_weights=True  # Restores the weights of the best epoch
    )
    callbacks = {
    'early_stopping': early_stopping_monitor}    

grid_search.fit(X_train, y_train_encoded, validation_data=(X_test,y_test_encoded), callbacks=callbacks)

# Print the best parameters and score
print("Best parameters: ", grid_search.bdef Keras_Classifier(build_fn=create_FCNNsmodel):
    # Wrap the TensorFlow model using KerasClassifier
    keras_model = KerasClassifier(build_fn=build_fn)
    # Define the hyperparameter grid for the grid search
    param_grid = {
    'batch_size': [8,10,16],
    'epochs': [90,95,100,105,110,120]   
     }

    # Perform grid search with cross-validation and include validation data and callbacks
    grid_search = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=5) 
    
    early_stopping_monitor = EarlyStopping(
    monitor='val_loss',  # Metric to monitor (e.g., validation loss)
    patience=3,  # Number of epochs to wait for improvement
    restore_best_weights=True  # Restores the weights of the best epoch
    )
    callbacks = {
    'early_stopping': early_stopping_monitor}

    return  grid_search, callbacksest_params_)
print("Best score: ", grid_search.best_score_)

# Access the training and validation scores for each parameter combination
                       
cv_results = grid_search.cv_results_
#train_scores = cv_results['mean_train_score']
val_scores = cv_results['mean_test_score']

# Print the training and validation scores for each parameter combination
  
a=pd.DataFrame()
for val_score, params in zip( val_scores, cv_results['params']):
   #print(f" Validation Score: {val_score:.4f}, Params: {params}")                             
   b=pd.DataFrame(params,index=[0])
   b=b.assign( val_score=val_score) 
   a=pd.concat([a,b])

a=a.sort_values(by = 'val_score', ascending=False)
a
Soil_Export= 'C:/Users/user/Google Drive/TCD20/python/final_project/csv' 
a.to_csv(os.path.join(Soil_Export,'optimization of batch and epochs¶.csv'))


## optimization of pramter
## fit the model

In [ ]:
def create_FCNNsmodel3(activation='relu', num_neurons=16,  dropout=0.2, learning_rate=0.001):
    model = Sequential()
    model.add(Input(shape=(3,)))
    model.add(Dense(num_neurons, activation=activation))
    model.add(Dropout(rate=0.2))
    model.add(Dense(num_neurons, activation=activation))
    model.add(Dense(num_neurons, activation=activation))
    model.add(Dense(5, activation='softmax'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

keras_model = KerasClassifier(build_fn=create_FCNNsmodel3, batch_size=16, epochs=100)
# Define the hyperparameter grid for the grid search
param_grid = {
    'activation': ['relu', 'tanh', 'sigmoid', 'linear'],
    'num_neurons': [1, 5, 10,  20, 50],
    'learning_rate': [0.001, 0.01, 0.1, 0.5]
}




# Create an instance of the EarlyStopping callback
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',  # Metric to monitor (e.g., validation loss)
    patience=3,  # Number of epochs to wait for improvement
    restore_best_weights=True  # Restores the weights of the best epoch
)

# Perform grid search with cross-validation and include validation data and callbacks
grid_search = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=5,verbose=2)
grid_search.fit(X_train, y_train_encoded, validation_data=(X_test, y_test_encoded), callbacks=[early_stopping_monitor])




grid_search.fit(X_train, y_train_encoded, validation_data=(X_test, y_test_encoded))

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


cv_results = grid_search.cv_results_
#train_scores = cv_results['mean_train_score']
val_scores = cv_results['mean_test_score']

# Print the training and validation scores for each parameter combination
  
history_df=pd.DataFrame()
for val_score, params in zip( val_scores, cv_results['params']):
   #print(f" Validation Score: {val_score:.4f}, Params: {params}")                             
   b=pd.DataFrame(params,index=[0])
   b=b.assign( val_score=val_score) 
   history_df=pd.concat([history_df,b])

history_df=history_df.sort_values(by = 'val_score', ascending=False)
a
Soil_Export= 'C:/Users/user/Google Drive/TCD20/python/final_project/csv' 
history_df.to_csv(os.path.join(Soil_Export,'optimization.csv'))
history_df
 




### Summary model results

In [ ]:
# Obtain the best model from the grid search
best_model = grid_search.best_estimator_.model


labels=['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil']
labels=[0,1,2,3,4]

classification_report1,confusion_matrix1,multi_confusion_matrix1=model_predict(best_model,X_test,y_test,labels)
classification_report_df, dict_matrix, confusion_matrix_df=model_summary(classification_report1,multi_confusion_matrix1,confusion_matrix1,labels)
f=plot_model_results(history_df)
Soil_Export= 'C:/Users/user/Google Drive/TCD20/python/final_project/fig'       
f.savefig(os.path.join(Soil_Export,'soil_image_dl_optimization.png'))      


#classification_report_df
print('### Yellow Soil ###')
print(dict_matrix['Yellow Soil'])
print('### Laterite Soil	 ###')
print(dict_matrix['Laterite Soil'])
print('### Black Soil ###')
print(dict_matrix['Black Soil'])
print('### Cinder Soill	 ###')
print(dict_matrix['Cinder Soil'])
print('### Peat Soil	 ###')
print(dict_matrix['Peat Soil'])


print(confusion_matrix_df)
Soil_Export= 'C:/Users/user/Google Drive/TCD20/python/final_project/csv'   
classification_report_df.to_csv(os.path.join(Soil_Export,'soil_image_dl_optimization.csv'))
classification_report_df

# inport the image data

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir= 'C:/Users/user/Google Drive/TCD20/python/dataset/Soil types'


batch_size = 32
img_height = 180
img_width = 180

train_ds= tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)



class_names =train_ds.class_names
print(class_names)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds_predict = val_ds.map(lambda x, y: (x / 255.0, y))
y_train= []
for images, labels in train_ds:
    y_train.extend(labels.numpy())
y_train = np.array(y_train)


y_test = []
for images, labels in val_ds:
    y_test.extend(labels.numpy())
y_test = np.array(y_test)


 
    


# from  image   Convolutional Neural Networks (CNNs)



## Build the model
### Set up the layers

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import CategoricalCrossentropy,SparseCategoricalCrossentropy




def create_CNNmodel():                      
    model = Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(5,activation='softmax')
    ])
    return model



model=create_CNNmodel()
model.summary()

## Compile ane fit the model

In [ ]:
model=create_CNNmodel()

early_stopping_monitor = EarlyStopping(
    monitor='val_loss',  # Metric to monitor (e.g., validation loss)
    patience=3,  # Number of epochs to wait for improvement
    restore_best_weights=True  # Restores the weights of the best epoch
)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=20
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[early_stopping_monitor]  # Include the EarlyStopping callback  
)

history_df=pd.DataFrame(history.history)
history_df

### Summary model results

In [ ]:
labels=[0,1,2,3,4]
classification_report1,confusion_matrix1,multi_confusion_matrix1=model_predict(model,val_ds, y_test,labels)
classification_report_df, dict_matrix, confusion_matrix_df=model_summary(classification_report1,multi_confusion_matrix1,confusion_matrix1,labels)
f=plot_model_results(history_df)
Soil_Export= 'C:/Users/user/Google Drive/TCD20/python/final_project/fig'       
f.savefig(os.path.join(Soil_Export,'soil_image_dl_2.png'))      



print(classification_report_df)
print(confusion_matrix_df)
print(dict_matrix['Yellow Soil'])
Soil_Export= 'C:/Users/user/Google Drive/TCD20/python/final_project/csv'   
classification_report_df.to_csv(os.path.join(Soil_Export,'soil_image_dl_2.csv'))

classification_report_df



In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal_and_vertical",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    #tf.keras.layers.RandomCrop(
    #35,5, seed=12),

    layers.RandomRotation(0.5),
    layers.RandomZoom(0.5),
  ]
)

#data_augmentation = tf.keras.Sequential([
  #layers.RandomFlip("horizontal_and_vertical"),
  #layers.RandomRotation(0.1),
#])

resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(180, 180),
  layers.Rescaling(1./255)
])

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(22):
    print(_)
    for i in range(5):
     augmented_images = data_augmentation(images)
     ax = plt.subplot(3, 3, i + 1)
     plt.imshow(augmented_images[0].numpy().astype("uint8"))
     plt.axis("off")
   

In [ ]:
model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  data_augmentation,  
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(5)
])


model.summary()



In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history=model.summary()

epochs=20
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

history_df=pd.DataFrame(history.history)
history_df

In [ ]:
labels=[0,1,2,3,4]
classification_report1,confusion_matrix1,multi_confusion_matrix1=model_predict(model,val_ds, y_test,labels)
classification_report_df, dict_matrix, confusion_matrix_df=model_summary(classification_report1,multi_confusion_matrix1,confusion_matrix1,labels)
f=plot_model_results(history_df)
Soil_Export= 'C:/Users/user/Google Drive/TCD20/python/final_project/fig'       
f.savefig(os.path.join(Soil_Export,'soil_image_dl_3.png'))      



print(classification_report_df)
print(confusion_matrix_df)
print(dict_matrix['Yellow Soil'])
Soil_Export= 'C:/Users/user/Google Drive/TCD20/python/final_project/csv'   
classification_report_df.to_csv(os.path.join(Soil_Export,'soil_image_dl_3.csv'))
classification_report1